In [1]:
import numpy as np
import pandas as pd
import random
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

matrix = np.loadtxt('matrix.txt')
matrix = matrix.astype(int)
print(matrix)


[[ 0 -1  4 -1 -1 -1 -1 -1  3  1 -1  2 -1  5 -1 -1]
 [ 4  0 -1 -1  1 -1  5 -1 -1 -1  2 -1 -1 -1  3 -1]
 [-1 -1  0  4  2 -1 -1 -1 -1  3 -1  5 -1  1 -1 -1]
 [-1  3  2  0 -1 -1 -1 -1  1  4 -1 -1 -1 -1  5 -1]
 [ 3 -1 -1 -1  0 -1 -1 -1 -1  2 -1  1 -1  4 -1  5]
 [ 5 -1 -1 -1 -1  0 -1  1  4 -1  3 -1 -1 -1 -1 -1]
 [-1 -1 -1  5 -1  4  0 -1 -1 -1 -1  3  2 -1 -1  1]
 [ 1 -1 -1 -1  5 -1 -1  0  2 -1 -1 -1  4 -1 -1  3]
 [-1 -1  1 -1  3 -1 -1  4  0 -1  5 -1 -1 -1 -1  2]
 [-1  2  5 -1 -1 -1  1 -1 -1  0 -1  4 -1  3 -1 -1]
 [-1  5 -1  3  4 -1  2 -1 -1 -1  0 -1 -1 -1 -1 -1]
 [-1  4 -1  1 -1 -1 -1  3  5 -1 -1  0 -1  2 -1 -1]
 [-1  1 -1 -1 -1  3 -1  2 -1  5 -1 -1  0 -1  4 -1]
 [-1 -1 -1  2 -1  1  3 -1 -1 -1  4 -1  5  0 -1 -1]
 [-1 -1 -1 -1 -1  5 -1 -1 -1 -1 -1 -1  3 -1  0  4]
 [ 2 -1  3 -1 -1 -1  4  5 -1 -1 -1 -1  1 -1 -1  0]]


In [2]:
h, w = matrix.shape
possible_partners = [[] for _ in range(h)]
for i in range(h):
    inner_list = []
    for j in range(w):
        if matrix[i][j] == -1:
            inner_list.append(j)
    possible_partners[i] = inner_list

# Choose a partner that a person hasn't had before
def choose_partner(possible_partners, gifter):
    partner = random.choice(possible_partners[gifter])
    return partner

# Check to see if that person has already been used
def is_valid(matrix, reciever, year, tried):
    for i in range(0,16):
        if reciever in tried[i]:
            return False
        if matrix.T[reciever][i] == year:
            return False
        
    return True


In [3]:
tried = [[] for _ in range(h)]

def assign_partners(matrix, possible_partners, year, tried, index=0):
    if index == len(matrix):
        return True  # All partners have been assigned
    random.shuffle(possible_partners[index])
    for reciever in possible_partners[index]:
        if is_valid(matrix, reciever, year, tried):
            matrix[index][reciever] = year
            tried[index].append(reciever)
            possible_partners[index].remove(reciever)

            if assign_partners(matrix, possible_partners, year, tried, index + 1):
                return True  # Recurse

            matrix[index][reciever] = -1  # Undo assignment
            tried[index].remove(reciever)  # Undo assignment
            possible_partners[index].append(reciever)

    return False
index = 0
remaining_years = [i for i in range(6,16)]
for year in remaining_years:
    assign_partners(matrix, possible_partners, year, tried, index)

print(matrix)

[[ 0 -1  4 -1 -1 -1 -1  6  3  1 -1  2 -1  5 -1 -1]
 [ 4  0 -1  6  1 -1  5 -1 -1 -1  2 -1 -1 -1  3 -1]
 [-1 -1  0  4  2 -1 -1 -1 -1  3 -1  5 -1  1  6 -1]
 [ 6  3  2  0 -1 -1 -1 -1  1  4 -1 -1 -1 -1  5 -1]
 [ 3  6 -1 -1  0 -1 -1 -1 -1  2 -1  1 -1  4 -1  5]
 [ 5 -1  6 -1 -1  0 -1  1  4 -1  3 -1 -1 -1 -1 -1]
 [-1 -1 -1  5  6  4  0 -1 -1 -1 -1  3  2 -1 -1  1]
 [ 1 -1 -1 -1  5 -1 -1  0  2 -1  6 -1  4 -1 -1  3]
 [-1 -1  1 -1  3 -1 -1  4  0  6  5 -1 -1 -1 -1  2]
 [-1  2  5 -1 -1 -1  1 -1 -1  0 -1  4 -1  3 -1  6]
 [-1  5 -1  3  4 -1  2 -1 -1 -1  0 -1  6 -1 -1 -1]
 [-1  4 -1  1 -1  6 -1  3  5 -1 -1  0 -1  2 -1 -1]
 [-1  1 -1 -1 -1  3 -1  2  6  5 -1 -1  0 -1  4 -1]
 [-1 -1 -1  2 -1  1  3 -1 -1 -1  4  6  5  0 -1 -1]
 [-1 -1 -1 -1 -1  5  6 -1 -1 -1 -1 -1  3 -1  0  4]
 [ 2 -1  3 -1 -1 -1  4  5 -1 -1 -1 -1  1  6 -1  0]]


In [4]:
print(possible_partners)

[[10, 12, 4, 15, 14, 1, 6, 5, 3], [13, 9, 11, 15, 8, 12, 5, 7, 2], [7, 1, 5, 10, 8, 15, 0, 6, 12], [6, 11, 4, 10, 13, 12, 15, 5, 7], [7, 5, 10, 8, 14, 3, 2, 12, 6], [3, 15, 13, 14, 1, 4, 6, 9, 12, 11], [7, 2, 0, 14, 10, 13, 8, 9, 1], [13, 9, 2, 11, 5, 6, 1, 3, 14], [6, 11, 0, 12, 5, 14, 1, 13, 3], [7, 0, 14, 12, 3, 10, 5, 8, 4], [9, 13, 2, 14, 0, 11, 5, 15, 8, 7], [10, 12, 15, 6, 2, 0, 4, 9, 14], [13, 10, 3, 4, 15, 2, 6, 0, 11], [2, 7, 9, 15, 4, 8, 14, 1, 0], [11, 2, 7, 10, 4, 0, 3, 8, 9, 1, 13], [14, 9, 4, 10, 5, 11, 3, 1, 8]]


In [5]:
# Run this cell to save the current matrix to a text file 
# to use for the next year

np.savetxt('matrix_year_6.txt', matrix, fmt='%d')

In [9]:
name_dictionary = {0:"Logan", 1:"Matt", 2:"Jake", 3:"Kyle", 4:"Manny", 5:"Brett", 6:"Garrett", 7:"Steve", 8:"Nicky",
                  9:"Joel", 10:"Nico", 11:"Will", 12:"Paul", 13:"Riley", 14:"Ben", 15:"Samie"}
email_dictionary = {0:"#", 1:"#", 2:"#", 3:"#",
                   4:"#", 5:"#", 6:"#",
                   7:"#", 8:"#", 9:"#", 10:"#",
                   11:"#", 12:"#", 13:"#", 14:"#",
                   15:"#"}

password = "#"
#msg['From'] = "logansingerman@gmail.com"
server = smtplib.SMTP('smtp.gmail.com',587)
server.starttls()
server.login("#", password)

current_year = 6
# Put this in for loop to send all emails
for i in range(0,16):
    for j in range(0,16):
        if matrix[i][j] == current_year:
            msg = MIMEMultipart()
            msg['From'] = "#"
            msg['To'] = email_dictionary[i]
            msg['Subject'] = "Secret Santa"
            person = name_dictionary[j]
            message = f"You have {person} for secret santa"
            msg.attach(MIMEText(message, 'plain'))
            server.sendmail(msg['From'], msg['To'], msg.as_string())
server.quit()
            
            


(221,
 b'2.0.0 closing connection t5-20020ab04ac5000000b007ab975b8eb9sm1308403uae.10 - gsmtp')